### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [1]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [2]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [3]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

# สร้าง Xtrain_tfidf
Xtrain_tfidf = tfidf.fit_transform(Xtrain)

# สร้าง Xtest_tfidf
Xtest_tfidf = tfidf.transform(Xtest)


In [8]:
#tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [19]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=5)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier()

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [22]:
from sklearn.metrics import classification_report

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)

#report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.18      0.48      0.26        21
           1       0.38      0.48      0.43        21
           2       0.56      0.58      0.57        26
           3       0.52      0.50      0.51        34
           4       0.67      0.53      0.59        34
           5       0.88      0.54      0.67        26
           6       0.50      0.41      0.45        22
           7       0.58      0.54      0.56        28
           8       0.74      0.70      0.72        33
           9       0.82      0.72      0.77        25
          10       0.75      0.56      0.64        27
          11       0.71      0.85      0.77        20
          12       0.31      0.21      0.25        24
          13       0.79      0.48      0.59        23
          14       0.76      0.57      0.65        28
          15       0.67      0.55      0.60        29
          16       0.56      0.67      0.61        21
          17       0.59    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [23]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfVectorizer()),
    ('knn', KNeighborsClassifier(n_neighbors=5))])

# Fit the pipeline to the training set

pipeline.fit(Xtrain, ytrain)
# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.18      0.48      0.26        21
           1       0.38      0.48      0.43        21
           2       0.56      0.58      0.57        26
           3       0.52      0.50      0.51        34
           4       0.67      0.53      0.59        34
           5       0.88      0.54      0.67        26
           6       0.50      0.41      0.45        22
           7       0.58      0.54      0.56        28
           8       0.74      0.70      0.72        33
           9       0.82      0.72      0.77        25
          10       0.75      0.56      0.64        27
          11       0.71      0.85      0.77        20
          12       0.31      0.21      0.25        24
          13       0.79      0.48      0.59        23
          14       0.76      0.57      0.65        28
          15       0.67      0.55      0.60        29
          16       0.56      0.67      0.61        21
          17       0.59    

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [30]:
from sklearn.model_selection import GridSearchCV

params = {
    'tfidf__max_features': [3000],
    'tfidf__ngram_range': [(1,1)],
    'knn__n_neighbors': [3, 5],
    'knn__weights': ['uniform']
}

gridcv = GridSearchCV(
    pipeline,
    params,
    scoring='f1_macro',
    cv=3,
    n_jobs=-1
)

gridcv.fit(Xtrain, ytrain)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [3, 5],
                         'knn__weights': ['uniform'],
                         'tfidf__max_features': [3000],
                         'tfidf__ngram_range': [(1, 1)]},
             scoring='f1_macro')

In [31]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=3000)),
                ('knn', KNeighborsClassifier())])

In [34]:
# Make predictions on the test set
y_pred = best_model.predict(Xtest)

# Report classification scores
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.52      0.37        21
           1       0.25      0.52      0.34        21
           2       0.38      0.38      0.38        26
           3       0.48      0.47      0.48        34
           4       0.78      0.41      0.54        34
           5       0.62      0.50      0.55        26
           6       0.45      0.41      0.43        22
           7       0.53      0.57      0.55        28
           8       0.62      0.64      0.63        33
           9       0.73      0.76      0.75        25
          10       0.81      0.48      0.60        27
          11       0.75      0.75      0.75        20
          12       0.33      0.17      0.22        24
          13       0.50      0.39      0.44        23
          14       0.62      0.46      0.53        28
          15       0.59      0.59      0.59        29
          16       0.45      0.67      0.54        21
          17       0.62    

In [33]:
best_model = gridcv.best_estimator_


#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [39]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


# 1) Define parameter grid (เฉพาะ n_neighbors & metric)

params = {
    'knn__n_neighbors': [3, 5, 7, 9],
    'knn__metric': ['euclidean', 'manhattan', 'cosine']
}





In [40]:
# 2) Define GridSearchCV (5-fold, f1_macro)

gridcv = GridSearchCV(
    pipeline,
    params,
    scoring='f1_macro',
    cv=5,
    n_jobs=-1
)

In [41]:
# 3) Fit on the training set

gridcv.fit(Xtrain, ytrain)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__metric': ['euclidean', 'manhattan', 'cosine'],
                         'knn__n_neighbors': [3, 5, 7, 9]},
             scoring='f1_macro')

In [42]:
# 4) Get best model

best_model = gridcv.best_estimator_

In [43]:
# 5) Predict on test set

y_pred = best_model.predict(Xtest)

In [44]:
# 6) Compute f1_macro on test set

f1_macro_test = f1_score(ytest, y_pred, average='macro')


# 7) Print required answers

print("Best n_neighbors:", gridcv.best_params_['knn__n_neighbors'])
print("Best metric:", gridcv.best_params_['knn__metric'])
print("Test set F1-macro score:", f1_macro_test)

Best n_neighbors: 5
Best metric: euclidean
Test set F1-macro score: 0.5423793486154647


Best n_neighbors = 5

Best metric = euclidean

Test F1-macro score = 0.54